# Import data from excels into database

In [ ]:
import pyodbc
import pandas as pd
import os
from datetime import datetime
from src.etl_functions import *
import settings

## Daily data (2023-2024 format)
This format has persons on rows and days on columns. One table per month.

In [ ]:
query_folder = 'data/data_2024'

In [ ]:
# FILE MANAGEMENT #
year = int(query_folder.split('_')[1])
year_files = []
for file in os.listdir(query_folder):
    if file.endswith('.csv') and len(file.split('_')) == 2:
        year_files.append(query_folder+'/'+file)

# CONNECTION TO THE DATABASE #
conn = settings.con
cursor = conn.cursor()

for monthly_path in year_files:
    data, month =load_month_data(monthly_path)

    # Transform the original dictionary
    preprocessed_dict = {}
    for person_id, counts in data.items():
        transformed_list = []
        dates_generator = generate_dates(month,year)
        for count in counts:
            date = next(dates_generator)
            transformed_list.extend([(date, str(i)) for i in range(1, count + 1)])
        preprocessed_dict[person_id] = transformed_list

    for person_id,tuple_list in preprocessed_dict.items():
        for tuple in tuple_list:
            date = tuple[0]
            instance = tuple[1]
            weekday = get_weekday(date)
            query = f"""INSERT INTO Gaioles ([Person_id],[Day],[Instance],[Weekday])
                        VALUES ('{person_id}', '{date}', '{instance}', '{weekday}')"""
            cursor.execute(query)
            con.commit()

## Hourly data (2025 format)
There is one table per person, days on columns and time periods on rows.

In [ ]:
csv_file = 'data_2025/Data_02_hourly.csv'

In [2]:
# Load CSV file in a format like 'data_2025/Data_01_hourly.csv'
df = pd.read_csv(csv_file, header=None)
month = csv_file.split("/")[1].split("_")[1]
year =  csv_file.split("/")[0].split("_")[1]

def process_csv(df):
    records = []
    person_id = None
    instance_tracker = {}  # Dictionary to track instance count per person & day
    
    for index, row in df.iterrows():
        first_col = str(row[0])

        if first_col.startswith("Total") or first_col == "NS" or first_col == 'nan':
            continue  # Skip irrelevant rows

        if len(first_col) in (3,4):  # Detect Person ID (e.g., 'RLP', 'HCT')
            person_id = first_col
            instance_tracker[person_id] = {}  # Reset instance tracking for new person
        
        elif "-" in first_col:  # Time period row (e.g., '00-03')
            time_period = first_col
            
            for day_idx, value in enumerate(row[1:], start=1):
                if pd.notna(value) and value > 0:  # Event occurred
                    date_str = f"{year}-{month}-{day_idx:02d}"
                    date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
                    weekday = date_obj.strftime("%A")

                    # Increase instance count per person & day
                    if day_idx not in instance_tracker[person_id]:
                        instance_tracker[person_id][day_idx] = value
                    else:
                        instance_tracker[person_id][day_idx] += value

                    instance = instance_tracker[person_id][day_idx]
                    records.append((person_id, date_obj, instance, weekday, time_period))
    
    return records


# Insert data into SQL Server
def insert_into_db(records):
    conn = settings.con
    cursor = conn.cursor()
    
    insert_query = """
    INSERT INTO Gaioles (Person_id, Day, Instance, Weekday, Time_period) 
    VALUES (?, ?, ?, ?, ?)
    """
    
    cursor.executemany(insert_query, records)
    conn.commit()

In [3]:
# Process CSV data
records = process_csv(df)

# Run the insertion
insert_into_db(records)

print("Data successfully inserted into the database.")

Data successfully inserted into the database.
